# Intro to Graph Databases

## A Database Container

![](images/DB_containers_internal.png)

In [70]:
%%bash
docker run \
    -d --name neo4j \
    --rm \
    --publish=7474:7474 \
    --publish=7687:7687 \
    --env NEO4J_AUTH=neo4j/class \
    neo4j

8956dd638832111b70b2b82890829f6fb5a5a10f85f695f83fee862801f7323f


In [71]:
%%bash
docker ps -a | grep neo4j

8956dd638832        neo4j                                                                                                                                "/docker-entrypoint.…"   4 seconds ago       Up 5 seconds              0.0.0.0:7474->7474/tcp, 7473/tcp, 0.0.0.0:7687->7687/tcp   neo4j


Navigate to http://localhost:7474/browser/ to work with a browser based Neo4j console:

![](./images/neo4j_login.png)

### A Simple Graph

The simplest graph has just a single node with some named values called Properties. 

Start by drawing a circle for the node. Add a name of the corresponding person, her job and her birthday.

  * **Nodes** are the name for data records in a graph
  * Data is stored as **Properties**
  * **Properties** are simple key/value pairs
  
  
![](./images/single_node.gv.svg)

###  Labels

Labels associate a set of nodes. Think of them as the type of your nodes.

Nodes can be grouped together by applying a Label to each member. In our social graph, we will label each node that represents a `Person`.

Apply the label `Person` to the node we created for *Odessa*

  * Color `Person` nodes green
  * A node can have zero or more labels
  * Labels do not have any properties

### More Nodes

Schema-free, nodes can have a mix of common and unique properties

Like any database, storing data in Neo4j can be as simple as adding more records. We will add a few more nodes:

  * Emil has a klout score of 99 (https://en.wikipedia.org/wiki/Klout)
  * Johan, from Sweden, who is learning to surf
  * Ian, from England, who is an author
  * Rik, from Belgium, has a cat named Orval
  * Allison, from California, who surfs
  * Similar nodes can have different properties
  * Properties can be strings, numbers, or booleans

Neo4j can store billions of nodes.

### Relationships

Relationships connect nodes in the graph. The real power of Neo4j is in connected data. To associate any two nodes, add a Relationship which describes how the records are related.

In our social graph, we simply say who `KNOWS` whom:

  * Emil `KNOWS` Johan and Ian
  * Johan `KNOWS` Ian and Rik
  * Rik and Ian `KNOWS` Allison
  * Relationships always have direction **OBS** That is, we model directed graphs in Neo4J.
  * Relationships always have a type
  * Relationships form patterns of data
  
![](images/simple_graph.gv.svg)

### Relationship properties

Store information shared by two nodes.

In a property graph, relationships are data records that can also contain properties. Looking more closely at Emil's relationships, note that:

  * Emil has known Johan since 2001
  * Emil rates Ian 5 (out of 5)
  * Everyone else can have similar relationship properties


![](images/simple_graph.gv.svg)

## Intro to Cypher

*Cypher* is Neo4j's graph query language. It is purpose built for working with graph data.

It makes it easy to work with graphs as it uses:
  
  * patterns to describe graph data
  * familiar SQL-like clauses

*Cypher* is a declarative language. That is, your patterns describe **what** to find, not **how** to find it.



### Creating Nodes

Let's use Cypher to generate a small social graph.

In [72]:
CREATE (a:Person { name: "Emil", from: "Sweden", klout: 99 })

That is it. The `CREATE` clause cossesponds to you drawing a circle on the whiteboard. You use it to create data, where:

  * `()` parenthesis indicate a node
  * `a:Person` a variable `a` and label `Person` for the new node
  * `{}` brackets add properties to the node

## Finding nodes

To find the node representing Emil:

In [75]:
MATCH (a:Person) 
WHERE a.name = "Emil" 
RETURN a

+--------------------------------------------------------------+
| a                                                            |
+--------------------------------------------------------------+
| (:Person {name: "Emil", _id_: 0, from: "Sweden", klout: 99}) |
+--------------------------------------------------------------+

1 row available after 1 ms, consumed after another 1 ms

The `MATCH` clause specifies a pattern of nodes and relationships.

  * `(a:Person)` a single node pattern with label 'Person' which will assign matches to the variable 'a'
  * `WHERE` clause to constrain the results
  * `a.name = "Emil"` compares name property to the value `"Emil"`
  * `RETURN` clause used to request particular results

To find all nodes in your database you would relax your query and just search for example for all nodes:

In [77]:
MATCH (a)
RETURN a;

+--------------------------------------------------------------+
| a                                                            |
+--------------------------------------------------------------+
| (:Person {name: "Emil", _id_: 0, from: "Sweden", klout: 99}) |
+--------------------------------------------------------------+

1 row available after 2 ms, consumed after another 0 ms

### Counting Nodes

In [ ]:
MATCH (a) 
RETURN count(*);

### Create More Nodes and Relationships

In [78]:
CREATE (js:Person { name: "Johan", from: "Sweden", learn: "surfing" })

In [80]:
MATCH (z:Person)
RETURN id(z);

+-------+
| id(z) |
+-------+
| 0     |
| 20    |
+-------+

2 rows available after 13 ms, consumed after another 1 ms

After creating two nodes -one for Emil and one for Johan respectively- manually, we have two disconnected nodes in the dataset. You could add a `KNOWS` relationship between them by matching the nodes and creating a new relationship.

In [81]:
MATCH (a),(b)
WHERE a.name = "Emil" AND b.name = "Johan"
CREATE (a)-[:KNOWS {since: 2001}]->(b)

However, the `CREATE` clause can create many nodes and relationships at once too.

In [82]:
MATCH (a:Person),(b:Person) 
WHERE a.name = "Emil" AND b.name = "Johan"
CREATE (ir:Person { name: "Ian", from: "England", title: "author" }),
(rvb:Person { name: "Rik", from: "Belgium", pet: "Orval" }),
(ally:Person { name: "Allison", from: "California", hobby: "surfing" }),
(a)-[:KNOWS {rating: 5}]->(ir),(b)-[:KNOWS]->(ir),(b)-[:KNOWS]->(rvb),
(ir)-[:KNOWS]->(b),(ir)-[:KNOWS]->(ally),(rvb)-[:KNOWS]->(ally)

In [83]:
MATCH (a)
RETURN a;

+-----------------------------------------------------------------------------+
| a                                                                           |
+-----------------------------------------------------------------------------+
| (:Person {name: "Emil", _id_: 0, from: "Sweden", klout: 99})                |
| (:Person {name: "Johan", _id_: 20, from: "Sweden", learn: "surfing"})       |
| (:Person {name: "Ian", _id_: 21, from: "England", title: "author"})         |
| (:Person {name: "Rik", _id_: 22, from: "Belgium", pet: "Orval"})            |
| (:Person {name: "Allison", _id_: 23, from: "California", hobby: "surfing"}) |
+-----------------------------------------------------------------------------+

5 rows available after 1 ms, consumed after another 1 ms

### How to see the entire graph?

In [84]:
MATCH (a:Person)-[r:KNOWS]-(b:Person)
RETURN a, r, b

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| a                                                                           | r                                     | b                                                                           |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| (:Person {name: "Emil", _id_: 0, from: "Sweden", klout: 99})                | [:KNOWS {rating: 5, _id_: 20}[0>21]]  | (:Person {name: "Ian", _id_: 21, from: "England", title: "author"})         |
| (:Person {name: "Emil", _id_: 0, from: "Sweden", klout: 99})                | [:KNOWS {_id_: 0, since: 2001}[0>20]] | (:Person {name: "Johan", _id_: 20, from: "Sweden", learn: "surfing"})       |
| (:Person

### Pattern Matching

Patterns -think of them as ASCII art representation of patterns in the graph- describe *what* to find in the graph. For instance, a pattern can be used to find Emil's friends:

In [85]:
MATCH (a:Person)-[r:KNOWS]-(friends)
WHERE a.name = "Emil" 
RETURN a, r, friends

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| a                                                            | r                                     | friends                                                               |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| (:Person {name: "Emil", _id_: 0, from: "Sweden", klout: 99}) | [:KNOWS {rating: 5, _id_: 20}[0>21]]  | (:Person {name: "Ian", _id_: 21, from: "England", title: "author"})   |
| (:Person {name: "Emil", _id_: 0, from: "Sweden", klout: 99}) | [:KNOWS {_id_: 0, since: 2001}[0>20]] | (:Person {name: "Johan", _id_: 20, from: "Sweden", learn: "surfing"}) |
+------------------------------------------------------------------------------------------------------------------

  * `MATCH` clause to describe the pattern from known Nodes to found Nodes
  * `(a)` starts the pattern with a Person (qualified by WHERE)
  * `-[:KNOWS]-` matches `KNOWS` relationships (in either direction)
  * (friends)will be bound to Emil's friends

#### How many friends does Emil have?

In [88]:
MATCH (a:Person)-[r:KNOWS]-(friends)
WHERE a.name = "Emil" 
RETURN a,r,friends

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| a                                                            | r                                     | friends                                                               |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| (:Person {name: "Emil", _id_: 0, from: "Sweden", klout: 99}) | [:KNOWS {rating: 5, _id_: 20}[0>21]]  | (:Person {name: "Ian", _id_: 21, from: "England", title: "author"})   |
| (:Person {name: "Emil", _id_: 0, from: "Sweden", klout: 99}) | [:KNOWS {_id_: 0, since: 2001}[0>20]] | (:Person {name: "Johan", _id_: 20, from: "Sweden", learn: "surfing"}) |
+------------------------------------------------------------------------------------------------------------------

### Recommendations

Using patterns and pattern matching we can quickly generate recommendations. For example, Johan is learning to surf, so he may want to find a new friend who already does:



In [ ]:
MATCH (a:Person)-[:KNOWS]-()-[:KNOWS]-(surfer)
WHERE a.name = "Johan" AND surfer.hobby = "surfing"
RETURN DISTINCT surfer

  * `()` empty parenthesis to ignore these nodes
  * `DISTINCT` because more than one path will match the pattern
  * `surfer` will contain Allison, a friend of a friend who surfs

### Deleting Nodes and Relations


You can delete nodes with the help of the `DELETE` directive.

```cypher
MATCH (a)
DELETE a
```

However, you cannot delete nodes, which are still connected with relations. That will result in an error message similar to the following:

```
Cannot delete node<512>, because it still has relationships. To delete this node, you must first delete its relationships.
```

Consequently, either delete the relations first or do both steps in one query

```cypher
MATCH (a)-[r]-(b)
DELETE a,b,r
```

In [ ]:
MATCH (n)
DETACH DELETE n

# Your turn!

Connect to the Neo4J web client and **read** and execute the movie graph tutorial

```
:play movie graph
```

Subsequently, create queries, which answer the following questions:

  1. How many movies from the 90ies are in our database and what are their names?
  * How many movies between 2000 and 2010 are in our database and what are their names?
  * Who produced "V for Vendetta"?
  * Which movies were directed by "Lana Wachowski"?
  * In which movies acted "Carrie-Anne Moss"?
  * Who were coactors of "Carrie-Anne Moss"?
  * In which way are people related to the movie "V for Vendetta"?
  * What is the shortest path between "Carrie-Anne Moss" and "Natalie Portman"?
  * With which other persons did "Natalie Portman" already work together? 
  * Create a recommendation to find actors with which "Natalie Portman" never worked together but her former colleagues did.

In [89]:
CALL db.schema()

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| nodes                                                                                                                                  | relationships                                                                                                                                                                                                      |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

##### Student Solutions

1. How many movies from the 90ies are in our database and what are their names?

```cypher
MATCH (nineties:Movie) WHERE nineties.released >= 1990 AND nineties.released < 2000 RETURN nineties.title
```

```cypher
MATCH (a:Movie) WHERE a.released >= 1990 AND a.released <= 1999 return collect(a.title), count(a)
```

```cypher
MATCH (nineties:Movie) WHERE nineties.released >= 1990 AND nineties.released < 2000 RETURN count(nineties);
```

```cypher
match (a:Movie) where a.released > 1989 and a.released < 2000 return a.title;
```


2. How many movies between 2000 and 2010 are in our database and what are their names?

```cypher
MATCH (nineties:Movie) WHERE nineties.released > 2000 AND nineties.released < 2010 RETURN nineties.title
```

```cypher
MATCH (a:Movie) WHERE a.released >= 2000 AND a.released <= 2010 return collect(a.title), count(a)
```

```cypher
match (a:Movie) where a.released > 1999 and a.released < 2011 return a,count(a)
```

3. Who produced "V for Vendetta"?

```cypher
MATCH (v {title: "V for Vendetta"})<-[:PRODUCED]-(producers) RETURN producers.name
```

```cypher
MATCH (title {title: "V for Vendetta"})<-[:PRODUCED]-(producers) RETURN producers.name
```

```cypher
MATCH (a:Person)-[:PRODUCED]->(b:Movie) WHERE b.title ="V for Vendetta" RETURN a.name
```

```cypher
MATCH (m:Movie)-[:PRODUCED]-(producer) WHERE m.title = "V for Vendetta" RETURN producer
```

```cypher    
MATCH (director:Person)-[:DIRECTED]->(movie:Movie)WHERE movie.title = "V for Vendetta" RETURN director;
```

4. Which movies were directed by "Lana Wachowski"?

```cypher
MATCH (p {name:"Lana Wachowski"})-[:DIRECTED]->(movie) RETURN movie.title
```

```cypher
MATCH (director:Person {name: "Lana Wachowski"})-[:DIRECTED]->(movies) RETURN movies.title
```

```cypher
MATCH (a:Person)-[:DIRECTED]->(b:Movie) WHERE a.name ="Lana Wachowski" RETURN b.title
```

```cypher
MATCH (director:Person)-[r:DIRECTED]->(movie:Movie) WHERE director.name = "Lana Wachowski" RETURN movie.title;
```

```cypher
MATCH (x:Person)-[:DIRECTED]->(m:Movie) WHERE x.name = "Lana Wachowski" RETURN m
```

5. In which movies acted "Carrie-Anne Moss"?

```cypher
MATCH (p {name:"Carrie-Anne Moss"})-[:ACTED_IN]->(movie) RETURN movie.title
```

```cypher
MATCH (a:Person)-[:ACTED_IN]->(b:Movie) WHERE a.name ="Carrie-Anne Moss" return b.title
```

```cypher
MATCH (carrie:Person {name: "Carrie-Anne Moss"})-[:ACTED_IN]->(movies) RETURN movies.title
```

```cypher
MATCH (actor:Person)-[r:ACTED_IN]->(movie:Movie) WHERE actor.name = "Carrie-Anne Moss" RETURN movie.title;
```

```cypher
MATCH (x:Person)-[:ACTED_IN]->(m:Movie) WHERE x.name = "Carrie-Anne Moss" RETURN m
```

6. Who were coactors of "Carrie-Anne Moss"?

```cypher
MATCH (p {name:"Carrie-Anne Moss"})-[:ACTED_IN]->()<-[:ACTED_IN]-(coact) RETURN coact.name
```

```cypher
MATCH (a:Person)-[:ACTED_IN]-(b:Movie)<-[:ACTED_IN]-(c:Person) WHERE c.name = "Carrie-Anne Moss" return a.name
```

```cypher
MATCH (carrie:Person {name:"Carrie-Anne Moss"})-[:ACTED_IN]->(movies)<-[:ACTED_IN]-(coActors) RETURN DISTINCT coActors.name
```

```cypher
MATCH (x:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(co:Person) WHERE x.name = "Carrie-Anne Moss" RETURN co
```

7. In which way are people related to the movie "V for Vendetta"?

```cypher
MATCH (people:Person)-[relatedTo]-(v:Movie {title: "V for Vendetta"}) RETURN v, people, Type(relatedTo), relatedTo
```

```cypher
MATCH (a:Person)-[r]->(b:Movie) WHERE b.title ="V for Vendetta" return a.name,type(r)
```

```cypher
MATCH (people:Person)-[relatedTo]-(:Movie {title: "V for Vendetta"}) RETURN DISTINCT people.name, Type(relatedTo)
```

```cypher
MATCH (x:Movie)-[r]-() WHERE x.title ="V for Vendetta" RETURN DISTINCT Type(r)
```


8. What is the shortest path between "Carrie-Anne Moss" and "Natalie Portman"?

```cypher
MATCH p=shortestPath(
  (carrie:Person {name:"Carrie-Anne Moss"})-[*]-(nat:Person {name:"Natalie Portman"})
)
RETURN p
```

```cypher
MATCH (Carrie:Person { name: 'Carrie-Anne Moss' }),(Natalie:Person { name: "Natalie Portman" }), p = shortestPath((Carrie)-[*..15]-(Natalie))
RETURN p
```

```cypher
MATCH p=shortestPath(
  (carrie:Person {name:"Carrie-Anne Moss"})-[*]-(nat:Person {name:"Natalie Portman"})
)
RETURN p
```

```cypher
MATCH p=shortestPath((C:Person {name:"Carrie-Anne Moss"})-[*]-(D:Person{name:"Natalie Portman"})) RETURN p
```

```cypher
MATCH p = shortestpath((natalie:Person {name: "Natalie Portman"})-[r *1..5]-(carrie:Person {name: "Carrie-Anne Moss"})) RETURN collect(p), length(p), natalie, r, carrie;
```

9. With which other persons did "Natalie Portman" already work together?

```cypher
MATCH (p {name:"Natalie Portman"})-[]->()<-[]-(coact) RETURN coact.name
```

```cypher
MATCH (p1 {name:"Natalie Portman"})-[]->()<-[]-(p2) RETURN p2
```

```cypher
MATCH (actor:Person)-[:ACTED_IN]->(m:Movie)-[r]-(coactor:Person) WHERE actor.name = "Natalie Portman" RETURN DISTINCT coactor;
```

```cypher
MATCH (x:Person {name:"Natalie Portman"})-[]->(:Movie)<-[]-(coWorkers:Person) RETURN coWorkers
```


10. Create a recommendation to find actors with which "Natalie Portman" never worked together but her former colleagues did.

```cypher
MATCH (p1 {name: "Natalie Portman"})-[]->()<-[]-(p2)-[]->()<-[]-(p3) where not (p1)-[]->()<-[]-(p3) RETURN distinct p3.name
MATCH (p1:Person {name: "Natalie Portman"})-[]->(:Movie)<-[]-(p2:Person)-[]->(:Movie)<-[]-(p3:Person) where not (p1)-[]->(:Movie)<-[]-(p3) RETURN distinct p3.name
```

```cypher
MATCH (nat:Person {name:"Natalie Portman"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors),
      (coActors)-[:ACTED_IN]->(m2)<-[:ACTED_IN]-(cocoActors)
WHERE NOT (nat)-[:ACTED_IN]->()<-[:ACTED_IN]-(cocoActors) AND nat <> cocoActors
RETURN cocoActors.name AS Recommended
```

```cypher
MATCH (a:Person)-[:ACTED_IN]->(b:Movie)-[:ACTED_IN]-(coa:Person)-[:ACTED_IN]->(coab:Movie)-[:ACTED_IN]-(ccoa:Person) WHERE         a.name = "Natalie Portman" RETURN DISTINCT ccoa
```

```cypher    
MATCH (p:Person {name:"Natalie Portman"})-[]->(:Movie)<-[]-(co:Person), (co)-[]->(:Movie)<-[]-(coco)
WHERE NOT (p)-[]->(:Movie)<-[]-(coco) AND p <> coco
RETURN coco.name
```

## What is a Graph?

### Undirected Graphs

![](./images/graph_example_small.png)

A *graph* $G$ consists of a nonempty set, $V(G)$, called the *vertices* of $G$, and a set $E(G)$ called the *edges* of $G$. An element of $V(G)$ is called a *vertex*. A vertex is also called a *node*; the words "vertex" and "node" are used interchangeably. An element of $E(G)$ is an *undirected* edge or simply an "edge". An undirected edge has two vertices $u\neq v$ called its endpoints. Such an edge can be represented by the two element set $\{u, v\}$. The notation $\langle u—v \rangle$ denotes this edge.
Both $\langle u—v \rangle$ and $\langle v—u \rangle$ define the same undirected edge, whose endpoints are $u$ and $v$.

![](./images/graph_example_small.png)

For example, let $H$ be the graph pictured in Figure above. The vertices of $H$
correspond to the nine dots, that is:

$V(H) = \{a,b,c,d,e,f,g,h,i\}$

The edges correspond to the eight lines, that is:

$E(H) = \big\{\langle a—b \rangle,\langle a—c \rangle,\langle b—d \rangle,\langle c—d \rangle,\langle c—e \rangle,\langle e—f \rangle,\langle e—g \rangle,\langle h—i \rangle\big\} $

### Directed Graphs

![](./images/digraph_example_small.png)

A *directed graph* -or *digraph*- $G$ consists of a nonempty set $V(G)$, called the vertices of $G$, and a set $E(G)$, called the edges of $G$. An element of $V(G)$ is called a *vertex*. A vertex is also called a *node*; the words "vertex" and "node" are used interchangeably. An element of $E(G)$ is called a *directed edge*. A directed edge is also called an "arrow" or simply an "edge". A directed edge starts at some vertex $u$ called the *tail* of the edge, and ends at some vertex $v$ called the *head* of the edge.

### Vertex Degrees

The *in-degree* of a vertex in a digraph is the number of arrows coming into it, and similarly its *out-degree* is the number of arrows out of it. More precisely,

If $G$ is a *digraph* and $v \in V(G)$, then 
    
  * $indeg(v) ::= \big\vert e \in \{E(G)\, \vert\, head(e) = v \}\big\vert$
  * $outdeg(v) ::= \big\vert e \in \{E(G)\, \vert\, tail(e) = v \}\big\vert$

### Walks and Paths


Picturing digraphs with points and arrows makes it natural to talk about following successive edges through the graph. For example, in the digraph above, you might start at vertex 1, successively follow the edges from vertex `a` to vertex `b`, from `b` to `c`, from `c` to `b`, and then from `b` to `d`. The sequence of edges followed in this way is called a *walk* through the graph. A *path* is a *walk* which never visits a vertex more than once. So following edges from `a` to `b` to `c` is a path, but it stops being a path if you go to `b` again.
The natural way to represent a walk is with the sequence of sucessive vertices it went through, in this case: 

$a\,b\,c\,b\,d$

However, it is conventional to represent a walk by an alternating sequence of successive vertices and edges, so this walk would formally be:

$a\, \langle a\rightarrow b \rangle \,b\, \langle b\rightarrow c \rangle \,c\, \langle c\rightarrow \,b\, \rangle \,b\, \langle b\rightarrow d \rangle\, d$


The redundancy of this definition is enough to make any computer scientist cringe, but it does make it easy to talk about how many times vertices and edges occur on the walk. Here is a formal definition:

A *walk* in a digraph is an alternating sequence of vertices and edges that begins with a vertex, ends with a vertex, and such that for every edge $\langle u\rightarrow v \rangle$ in the walk, vertex $u$ is the element just before the edge, and vertex $v$ is the next element after the edge.

So a walk $w$ is a sequence of the form

$w ::= v_{0}\, \langle v_{0}\rightarrow v_{1} \rangle \,v_{1}\, \langle v_{1}\rightarrow v_{2} \rangle \,v_{2}\, ... \langle v_{k-1}\rightarrow \,v_{k}\, \rangle \,v_{k}$

where $\langle v_{i}\rightarrow \,v_{i+1}\, \rangle \in E(G)$ for $i \in [0..k)$ The walk is said to start at $v_{0}$, to end at $v_{k}$, and the length $\vert v\vert$ of the walk is defined to be k.
The walk is a path if and only if all the $v_{i}$’s are different,that is,if $i \neq j$, then $v_{i} \neq v_{j}$.
A *closed* walk is a walk that begins and ends at the same vertex. A cycle is a positive length closed walk whose vertices are distinct except for the beginning and end vertices.


##### References:
The definitions above and the illustrations are taken from the book:
*Mathematics for Computer Science*, Eric Lehman, F. Tom Leighton, Albert R. Meyer
https://courses.csail.mit.edu/6.042/spring17/mcs.pdf